In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import pandas as pd
import numpy as np
import math
import collections
import statistics
import random
import itertools
import json

In [ ]:
G_AC = nx.read_graphml("../../data/graphs/G_final.graphml") 

In [ ]:
# Defining one function to obtain the edges which go from the studied genre (main genre) to others

def genre_connection(main_gen, N_G, G):
    MAIN_nod = [nod for nod,gen in N_G if gen == main_gen]
    to_out = 0
    to_main = 0
    conn = []

    for edge in G.edges():
        if (edge[0] in MAIN_nod) & (edge[1] not in MAIN_nod):
           conn.append(edge)
           to_out += 1

        elif (edge[0] not in MAIN_nod) & (edge[1] in MAIN_nod):
           conn.append(edge)
           to_out += 1
        
        elif (edge[0] in MAIN_nod) & (edge[1] in MAIN_nod):
            to_main += 1

    return(to_main, to_out, conn)

In [ ]:
# Defining one function to validate if the edges that go outside, match with certain genre (review genre)

def getratio_revgen(rev_gen, DICT):
    REV_rat = []
    for main,val in DICT.items():
        for elem in val.items():
            if elem[0]==rev_gen:
                REV_rat.append(elem[1])
    return(REV_rat)

In [ ]:
# Getting the degrees of the nodes in each Genre
nod_genre = sorted([(nod, G_AC._node[nod]['genre']) for nod in G_AC.nodes()], key=lambda x: x[1], reverse=True)
set_gen = set([G_AC._node[nod]['genre'] for nod in G_AC.nodes()])

dic_GenDeg = {}
for gen in set_gen:
    deg_g = [G_AC.degree[nod] for nod in G_AC.nodes() if G_AC._node[nod]['genre'] == gen]
    dic_GenDeg[gen] = deg_g

sort_vol = sorted([(gen, len(dic_GenDeg[gen])) for gen in set_gen], key=lambda x: x[1], reverse=True)
sort_gen = [gen for gen,n in sort_vol]

In [ ]:
# Iterating through all the genres to study their edges and obtain the ratios between genres

gen_it = sorted(sort_gen.copy())
GEN_RAT = {}
# GEN_RAT_DF = {}

for main_gen in gen_it:
    
    y1,y2,out_conn = genre_connection(main_gen, nod_genre, G_AC)
    tot = y1+y2
  
    gen_rev = sort_gen.copy()
    gen_rev.remove(main_gen)

    GEN_RATREV = {}
    GEN_RATREV[main_gen] = 100*y1/ tot

    for gen in gen_rev:
        REV_nod = [nod for nod,g in nod_genre if g == gen]
        to_grev = 0
        for edge in out_conn:
            if (edge[0] in REV_nod) or (edge[1] in REV_nod):
                to_grev += 1

        GEN_RATREV[gen] = 100*to_grev/tot
        # GEN_RATREV[gen] = math.ceil(100*to_grev/tot)
    
    od_GEN_RATREV = collections.OrderedDict(sorted(GEN_RATREV.items()))
    
    GEN_RAT[main_gen] = od_GEN_RATREV
    # GEN_RAT_DF[main_gen] = [i/100 for i in list(od_GEN_RATREV.values())]
    # GEN_RAT[main_gen] = GEN_RATREV
    # labels_DF = od_GEN_RATREV.keys()

In [ ]:
# Creating a dict with colors, to keep the same formatting 

color_state_nd = {
    "Rock": "rgb(65,105,225)",
    "Hip-Hop": "rgb(138,43,226)", 
    "Pop": "rgb(255,20,147)",
    "Alternative": "rgb(212,175,55)",
    "Country": "rgb(255,165,0)",
    "Funk-Soul": "rgb(0,191,255)",
    "Latin": "rgb(255,99,71)",
    "Punk": "rgb(34,139,34)",
    "Reggae": "rgb(144,238,144)",
    "New Age": "rgb(169,169,169)",
    "K-Pop": "rgb(211,211,211)"
}

In [ ]:
data = []
merged = []

In [ ]:
for main, val in GEN_RAT.items():
    genre = {}
    genre["genre"] = main
    for elem in val.items():
        genre[elem[0]] = elem[1]
        genre[elem[0] + "Color"] = color_state_nd[elem[0]]
    
    data.append(genre)

In [ ]:
# merged
for main, val in GEN_RAT.items():
    genre = {}
    genre["genre"] = main
    genre["Other"] = 0
    for elem in val.items():
        if elem[0] == main:
            genre[elem[0]] = elem[1]
            genre[elem[0] + "Color"] = color_state_nd[elem[0]]
        else:
            genre["Other"] += elem[1]
            genre["OtherColor"] = "#B0C4DE"
    
    merged.append(genre)

In [ ]:
with open("../../data/webpage/data-genres.json", "w") as outfile:
    json.dump(data, outfile)

In [ ]:
with open("../../data/webpage/data-genres-merged.json", "w") as outfile:
    json.dump(merged, outfile)